# TP2 - Equações do Movimento e Trajectórias

O trabalho da segunda aula prática é dedicado ao estudo do movimento de partículas sob potenciais clássicos como seja o potencial harmónico (massas unidas por molas). Os métodos e técnicas apresentadas neste tutorial/exercício são semelhantes às técnicas usadas em vários trabalhos de Modelação Molecular, no entanto, as expressões matemáticas que governam estes sistemas são bastante mais simples.



## Preparação do Ambiente Python

O _notebook_ usado como protocolo e folha de excercício deste trabalho é uma tecnologia que permite combinar texto formatado com código Python. Existem várias implementações desta tecnologia:
* **Jupyter Notebooks**: esta é provavelmente a implementação mais antiga do conceito de _notebook_. Encontra-se disponível para Windows, Mac e Linux em https://jupyter.org/ . Normalmente, o servidor do jupyter notebook é lançado na linha de comandos ou usando uma entrada no Finder ou menu Iniciar, e o utilizador deve usar um browser para navegar para o URL indicado de forma a poder abrir o notebook.
* **Jupyter Lab** é uma implementação mais recente e completa do jupyter notebook original. Tal como o jupyter notebook clássico, o programa em si funciona numa linha de comandos, e é acedido através de um URL que deve ser copiado para o browser (preferencialmente uma versão recente do Chrome, Chromium, ou Firefox).
* **Google Colab** é uma implementação de cálculo na _cloud_ desenvolvida pela Google e disponível em https://colab.research.google.com/ . Possui algumas vantagens relativamente ao uso de notebooks da Jupyter: os cálculos correm numa máquina remota da Google, não há nececidade de instalar software no computador local (basta um browser recente) e é gratuito, para o nível de acesso mais básico.

Este notebook deverá correr igualmente em qualquer destas três plataformas, mas as células de código abaixo poderão ter de ser executadas, dependendo da plataforma escolhida. Cada caso de utilização é apresentado individualmente. Deverá executar **apenas** a célula de código que corresponde à sua situação específica.

### Caso 1: Jupyter Lab nos computadores da ECUM

Os computadores da ECUM correm Windows num servidor e o perfil do utilizador é limpo no final da sessão. Por causa disso, a célula de código a seguir deve ser executada **uma vez após cada início de sessão no computador**.

In [ ]:
!pip install --user numpy
!pip install --user pandas
!pip install --user matplotlib

### Caso 2: Jupyter Lab ou Jupyter Notebook instalado com o Miniconda num computador pessoal
O gestor de pacotes miniconda instala uma versão minimalista do Python. A célula seguinte deve ser executada **apenas uma vez por utilizador**, já que os pacotes instalados permanecem disponíveis no ambiente do utilizador.

In [ ]:
!pip install --user numpy
!pip install --user pandas
!pip install --user matplotlib

### Caso 3: Jupyter Lab ou Jupyter Notebook numa instalação "normal" de Python num computador pessoal.

O instalador do Python apenas instala interpretador Python e os pacotes da biblioteca-padrão. Pacotes adicionais, tais como o `numpy` devem ser instalados à parte, usando o pip. Esta operação deve ser executadada **apenas uma vez por utilizador**, tal como no caso acima.

In [ ]:
!pip install --user numpy
!pip install --user pandas
!pip install --user matplotlib

### Caso 4: Google Colab

O ambiente oferecido pelo Google Colab ja inclui vários pacotes necessários para cálculo numérico, gráficos e tratamento de dados. Para este trabalho específico, não é necessário fazer a instalação de quaisquer pacotes adicionais.

### Caso 5: Jupyter Lab ou Jupyter Notebook instalado com o Anaconda (completo) num computador pessoal
O gestor de ambiente Anaconda normalmente instala os pacotes mais usados, pelo que não será necessário fazer a instalação de pacotes adicionais para este trabalho específico.

## Importar pacotes e carregar algumas funções utilitárias

Depois de configurar o seu ambiente python, deverá executar a célcula seguinte, de forma a carregar os pacotes necessários (instrução `import`), assim como definir algumas funções e constantes usadas ao longo do trabalho.

In [13]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# funcao para a o trabalho 1 pelo metodo de verlet
def work1_verlet(m, xi, k, x0, t_max, dt):
    omega = np.sqrt(k/m)
    print(f"Omega = {omega:10.5e} rad/s")
    print(f"Fequência = {omega/(2.0*np.pi):10.5e} Hz")
    #calcular o nº de pontos para o grafico
    npoints = int(np.ceil(t_max/dt))
    # Solucao analítica
    # guardamos os valores de x(t), v(t), V(t), Ec(t) e Etot(t) da solução analítica em vectores (arrays do numpy) com prefixo a_
    a_start = time.time()
    a_tempo = np.linspace(0,t_max,npoints)
    a_x_t = xi*np.cos(omega*a_tempo)
    a_v_t = -xi*omega*np.sin(omega*a_tempo)
    a_V_t = 0.5*k*((a_x_t-x0)**2)
    a_Ec_t = 0.5*m*(a_v_t**2)
    a_Et_t = a_V_t+a_Ec_t # energia total em função do tempo
    a_calculo = time.time() - a_start # tempo que demorou o calculo
    print(f"Tempo de calculo para a solução analítica: {a_calculo:0.3g} s")
    # Solucao numerica pelo metodo de verlet
    # guardamos os valores de x(t), v(t), V(t), Ec(t) e Etot(t) da solução analítica em vectores (arrays do numpy) com prefixo n_
    # no estado inicial, estas arrays estão cheias de zeros, excepto a primeira posição, na qual colocamos a posição, velocidade e aceleração iniciais.
    # Só depois do cálculo de x(t) e v(t) é que calculamos as energias, usando as mesmas expressões que para a solução analitica
    n_start = time.time()
    n_tempo = np.zeros(npoints)
    n_x_t = np.zeros(npoints)
    n_v_t = np.zeros(npoints)
    # colocar as condições iniciais
    n_x_t[0] = xi
    n_v_t[0] = 0.0
    # vamos agora iterar do ponto 1 até npoints. Cada passo é o ponto t+delta_t da exposição teórica
    for i in range(1,npoints):
        n_tempo[i] = n_tempo[i-1]+dt
        a=-k*(n_x_t[i-1]-x0)/m
        n_v_t[i] = n_v_t[i-1] + a*dt
        n_x_t[i] = n_x_t[i-1] + n_v_t[i-1]*dt + 0.5*a*(dt**2)
    n_V_t  = 0.5*k*((n_x_t-x0)**2)
    n_Ec_t = 0.5*m*(n_v_t**2)
    n_Et_t = n_V_t+n_Ec_t
    n_calculo = time.time() - n_start
    print(f"Tempo de calculo para a solução numérica: {n_calculo:0.3g} s")
    # gravar dados numa tabela
    dados = pd.DataFrame({'A_Tempo (s)':a_tempo,
                         'A_x (m)':a_x_t,
                         'A_v (m/s)':a_v_t,
                         'A_V (J)':a_V_t,
                         'A_K (J)':a_Ec_t,
                         'A_E (J)':a_Et_t,
                         'N_Tempo (s)':n_tempo,
                         'N_x (m)':n_x_t,
                         'N_v (m/s)':n_v_t,
                         'N_V (J)':n_V_t,
                         'N_K (J)':n_Ec_t,
                         'N_E (J)':n_Et_t,
                         })
    return(dados)
    


## Dois corpos unidos por uma mola

Este trabalho tem como objectivo estudar o movimento de dois corpos unidos por uma mola clássica. A força de uma mola depende da distância entre as suas extremidades $x$ e dois parâmetros da prórpia mola: a distância de equilíbrio $x_0$ e a constante da mola $k$. A energia potencial deste sistema é:
$$ V = (1/2) k (x-x_0)^2 $$ e a força excercida pela mola sobre cada corpo (Lei de Hooke) é $$ F = -k (x-x_0) $$ Se a massa de um dos corpos for muito superior à massa do segundo, o mais pesado pode ser considerado como estando parado. Se iniciarmos o nosso estudo com a partícula mais leve afastada da posição de equílibrio (denominemos a posição inicial por $x_i$) e sem velocidade inicial, o movimento deste corpo é dado pela equação: $$ x(t) = x_i cos(\omega t) $$ em que $$ \omega = \sqrt{k/m} $$ em que $m$ é a massa do corpo mais leve.

Nesta parte do trabalho, a trajectória deste corpo vai ser simulada usando um método numérico (algoritmo de Verlet) e comparada com a solução analítica apresentada acima. Ambos os métodos numéricos partem da expressão diferencial da lei de Hooke, combinada com a relação entre força e aceleração ($F=ma$): $$ m \frac{d^2x}{dt} = -k(x-x_0) $$.

A resolução analítica desta equação, para as condições iniciais descritas acima, leva ao seguinte conjunto de equações da posição ($x$), velocidade ($v$), energia cinética ($E_c$) e energia potencial ($V$) em função do tempo ($t$): $$ x(t) = x_i cos(\omega t) $$ $$ v(t) = -x_i \omega sen(\omega t)$$ $$ V(t) = (k/2) (x_i cos(\omega t) $$ e $$ E_c(t) = (\frac{-x_i m \omega}{2}) sen^2(\omega t) $$ Sendo a energia total to distema a soma de $E_c$ e $V$.

Já o método de Verlet considera a aceleração inicial $$a_0 = \frac{-k(x-x_0)}{m}$$ e a velocidade inicial $v_0 = 0$ para calcular a posição, velocidade e aceleração ao fim de um curto espaço de tempo $\Delta t$. Para uma sequência de pontos no tempo ($t_n$), a posição é determinada por $$ x(t+\Delta t) =  x(t) + v(t)\Delta t + (1/2)a(x(t))(\Delta t)^2$$ Sendo que a velocidade ($v(t)$) é calculada por propagção da velocidade e aceleração no ponto anterior: $$ v(t+\Delta t) = v(t) + a(x(t))\Delta t $$ E a acerelação é calculada a partir da força dada pela Lei de Hooke.

A célula abaixo corre uma simulação de um sistema de massa pressa por uma mola, usando o método de  Verlet e compara com a solução analítica dada acima. Os parâmetros da simulação são dados nas primeiras linhas da célula. Após execução da célula (`Shift+Enter` ou no botão `Run` da barra de ferramentas acima), o código devolve a velocidade angular ($\omega$) e a frequência de vibração do sistema, os tempos de cálculo para cada solução, assim como gráficos da variação da posição, velocidade, energia potencial, energia cinética e energia total ao longo do tempo da simulação. Também é feito o gráfico do espaço de fase Posição/Velocidade. No final também são dados valores médios e descios-padrão para cada uma das componentes da energia. Todos os dados da simulação podem ser gravados em formato csv para tratamento posterior no MS Excel ou LibreOffice Calc. **Deverá alterar os valores dos dados do sistema e da simulação numérica de forma a poder responder às questões colocadas abaixo** Durante este procedimento, tenha em atenção que:
* O tempo da simulação deverá ser ajustado de forma a cobrir 3 a 5 ciclos completos da trajectória.
* A diminuição abusiva do passo de integração pode levar ao bloqueio do seu sistema, com potencial perda de dados.
* A frequência da oscilação é calculada em função de $k$ e de $m$.
* Pode gravar cada linha de gráficos como imagens, clicando com o botão direito do rato e selecionando a opção `Save image as...`. A imagem pode depois ser inserida num documento word com as suas respostas, caso considere necessário.


In [ ]:
# Dados do sistema
massa = 0.05              # massa do corpo móvel, em kg
posicao_inicial = 1.0     # posição inicial, em m
k_mola = 2.0              # constante de força da mola, em N/m
k_x0   = 0.0              # comprimento de equilibrio da mola,  em m
# Dados da simulação numérica
tempo_maximo = 2.0        # tempo da simulação, em s
delta_t = 5.0e-3          # amplitude do passo de integração, em s
# Ficheiro onde gravar os dados da simulação (a terminação .csv é dada automaticamente)
nome_ficheiro = "simulação_1.1"

### não é necessario editar abaixo desta linha ###
### no need to edit past this line ###

dados_simul=work1_verlet(massa, posicao_inicial, k_mola, k_x0, tempo_maximo, delta_t)
# gravar os dados
dados_simul.to_csv(f"{nome_ficheiro}.csv", index=False)

# grafico das trajectórias
fig, ax = plt.subplots(ncols=3,figsize=(16,5))
ax[0].plot(dados_simul['A_Tempo (s)'],dados_simul['A_x (m)'], label="Solução Analítica")
ax[0].plot(dados_simul['N_Tempo (s)'],dados_simul['N_x (m)'], label="Verlet")
ax[0].set_xlabel("Tempo (s)")
ax[0].set_ylabel("Posição (m)")
ax[0].legend()
ax[1].plot(dados_simul['A_Tempo (s)'],dados_simul['A_v (m/s)'], label="Solução Analítica")
ax[1].plot(dados_simul['N_Tempo (s)'],dados_simul['N_v (m/s)'], label="Verlet")
ax[1].set_xlabel("Tempo (s)")
ax[1].set_ylabel("Velocidade (m/s)")
ax[1].legend()
ax[2].plot(dados_simul['A_x (m)'],dados_simul['A_v (m/s)'], label="Solução Analítica")
ax[2].plot(dados_simul['N_x (m)'],dados_simul['N_v (m/s)'], label="Verlet")
ax[2].set_xlabel("Posição (m)")
ax[2].set_ylabel("Velocidade (m/s)")
ax[2].legend()
plt.show()

# grafico das Energias
fig, ax = plt.subplots(ncols=3,figsize=(16,5))
ax[0].plot(dados_simul['A_Tempo (s)'],dados_simul['A_V (J)'], label="Solução Analítica")
ax[0].plot(dados_simul['N_Tempo (s)'],dados_simul['N_V (J)'], label="Verlet")
ax[0].set_xlabel("Tempo (s)")
ax[0].set_ylabel("Energia Potencial (J)")
ax[0].legend()
ax[1].plot(dados_simul['A_Tempo (s)'],dados_simul['A_K (J)'], label="Solução Analítica")
ax[1].plot(dados_simul['N_Tempo (s)'],dados_simul['N_K (J)'], label="Verlet")
ax[1].set_xlabel("Tempo (s)")
ax[1].set_ylabel("Energia Cinética (J)")
ax[1].legend()
ax[2].plot(dados_simul['A_Tempo (s)'],dados_simul['A_E (J)'], label="Solução Analítica")
ax[2].plot(dados_simul['A_Tempo (s)'],dados_simul['N_E (J)'], label="Verlet")
ax[2].set_xlabel("Posição (m)")
ax[2].set_ylabel("Energia Total (J)")
ax[2].legend()
plt.show()

print(f"{'Energia':^12s} {'Média (A)':^15s} {'Desvio-padrão (A)':^15s} {'Média (N)':^15s} {'Desvio-padrão (N)':^15s}")
print(f"{'Potencial':^12s} {dados_simul['A_V (J)'].mean():^15.3f} {dados_simul['A_V (J)'].std():^15.3f}  {dados_simul['N_V (J)'].mean():^15.3f} {dados_simul['N_V (J)'].std():^15.3f}")
print(f"{'Cinética':^12s} {dados_simul['A_K (J)'].mean():^15.3f} {dados_simul['A_K (J)'].std():^15.3f}  {dados_simul['N_K (J)'].mean():^15.3f} {dados_simul['N_K (J)'].std():^15.3f}")
print(f"{'Total':^12s} {dados_simul['A_E (J)'].mean():^15.3f} {dados_simul['A_E (J)'].std():^15.3f}  {dados_simul['N_E (J)'].mean():^15.3f} {dados_simul['N_E (J)'].std():^15.3f}")



### Questões

1. Corra o simulador com as seguintes condições (condições do ficheiro tal como é descarregado):
    * massa = 50 g
    * posição inicial = 1.0 m
    * constante da mola = 2.0 N/m
    * posição de equílibrio = 0.0 m
    * tempo da simulação: 2 s
    * passo de integração: 5.0e-3 s
1. Nas condições acima, como compara a evolução da posição ao longo do tempo dada pelo método de Verlet com a solução analítica.
1. Ainda nas mesmas condições, caracterize a evolução da energia total e das suas componentes (cinética e potencial) dadas pelo método de Verlet.
1. Mantendo as demais condições, como varia a divergência da energia total ($\Delta E / t_{simulação}$) com o passo de integração? Que conseguências tem esta variação no tempo de cálculo pelo método de Verlet (pode demonstar soba forma de um gráfico do tempo de cálculo em função do tamanho do passo)?
1. Mantendo as demais condições (e restaurando o passo de integração para o valor dado acima), como avalia a variação da energia total ao longo da trajéctória com a variação da posição inicial?
1. A frequência da oscilação harmónica depende da constante da mola e da massa da partícula. Altere qualquer um desses valores (ou ambos) de forma a alterar a frequência da oscilação para (aproximadamente) 0.5, 1 e 2 Hz. Anote as combinações massa/constante da mola escolhidas.
1. Para cada combinação escolhida no ponto anterior, encontre o tamanho máximo aceitável do passo de integração (considere como critério uma variação da energia total ao longo da simulação inferior a 5%). O que pode afirmar sobre a forma como o a frequência da oscilação afecta o passo de integração máximo aceitável?
1. (extra) Sugira uma estratégia possível para lidar com as divergências encontradas entre o método de Verlet e a solução analítica.